<a href="https://colab.research.google.com/github/RENISA-GRACELIN/Online_Internship_Projects/blob/main/Svm2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix

# Load the data
train_dir = '/content/drive/MyDrive/DR/Train'
test_dir = '/content/drive/MyDrive/DR/Test'
valid_dir = '/content/drive/MyDrive/DR/Valid'

train_datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True, zoom_range=0.2, brightness_range=[0.2,1.0])
test_datagen = ImageDataGenerator(rescale=1./255)

train_set = train_datagen.flow_from_directory(train_dir, target_size=(224, 224), batch_size=8, class_mode='categorical', shuffle=False)
valid_set = test_datagen.flow_from_directory(valid_dir, target_size=(224, 224), batch_size=8, class_mode='categorical', shuffle=False)
test_set = test_datagen.flow_from_directory(test_dir,target_size=(224, 224),batch_size=8,class_mode='categorical',shuffle=False
)
# Load the pre-trained model
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Make a new model that outputs the features from the pre-trained model
feature_extractor = Model(inputs=base_model.input, outputs=base_model.output)

# Extract features from the images
train_features = feature_extractor.predict(train_set)
valid_features = feature_extractor.predict(valid_set)
test_features = feature_extractor.predict(test_set)


Found 1750 images belonging to 5 classes.
Found 500 images belonging to 5 classes.
Found 250 images belonging to 5 classes.
 45/219 [=====>........................] - ETA: 2:06

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

# Define a pipeline with feature scaling and SVM classifier
svm_pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Scale features to have mean=0 and variance=1
    ('svm', SVC())  # SVM classifier
])

# Define the hyperparameters grid to search
param_grid = {
    'svm__C': [0.1, 1, 10, 100],  # Regularization parameter
    'svm__kernel': ['linear', 'poly', 'rbf'],  # Kernel type
    'svm__gamma': ['scale', 'auto']  # Kernel coefficient for 'rbf' and 'poly'
}

# Perform grid search with cross-validation
grid_search = GridSearchCV(svm_pipeline, param_grid, cv=5, verbose=2)
grid_search.fit(train_features_flat, train_labels)

# Get the best parameters
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

# Evaluate the model on the test set
y_pred = grid_search.predict(test_features_flat)

# Generate classification report
report = classification_report(test_labels, y_pred)
print("Classification Report:\n", report)


Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV] END ...svm__C=0.1, svm__gamma=scale, svm__kernel=linear; total time= 3.3min
[CV] END ...svm__C=0.1, svm__gamma=scale, svm__kernel=linear; total time= 3.3min
[CV] END ...svm__C=0.1, svm__gamma=scale, svm__kernel=linear; total time= 3.1min
[CV] END ...svm__C=0.1, svm__gamma=scale, svm__kernel=linear; total time= 3.2min
[CV] END ...svm__C=0.1, svm__gamma=scale, svm__kernel=linear; total time= 3.2min
[CV] END .....svm__C=0.1, svm__gamma=scale, svm__kernel=poly; total time= 3.4min
[CV] END .....svm__C=0.1, svm__gamma=scale, svm__kernel=poly; total time= 3.3min
[CV] END .....svm__C=0.1, svm__gamma=scale, svm__kernel=poly; total time= 3.3min
[CV] END .....svm__C=0.1, svm__gamma=scale, svm__kernel=poly; total time= 3.3min
[CV] END .....svm__C=0.1, svm__gamma=scale, svm__kernel=poly; total time= 3.4min
[CV] END ......svm__C=0.1, svm__gamma=scale, svm__kernel=rbf; total time= 3.9min
[CV] END ......svm__C=0.1, svm__gamma=scale, sv

In [ ]:

# Flatten the features for use with the classifiers
train_features_flat = np.reshape(train_features, (train_features.shape[0], -1))
valid_features_flat = np.reshape(valid_features, (valid_features.shape[0], -1))
test_features_flat = np.reshape(test_features, (test_features.shape[0], -1))

# Get the labels
train_labels = train_set.classes
valid_labels = valid_set.classes
test_labels = test_set.classes

# SVM
svm_classifier = svm.SVC()
svm_classifier.fit(train_features_flat, train_labels)
svm_predictions = svm_classifier.predict(test_features_flat)

# Print classification report
print("SVM Classification Report:")
print(classification_report(test_labels, svm_predictions))


SVM Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        50
           1       0.00      0.00      0.00        50
           2       0.00      0.00      0.00        50
           3       0.00      0.00      0.00        50
           4       0.20      1.00      0.33        50

    accuracy                           0.20       250
   macro avg       0.04      0.20      0.07       250
weighted avg       0.04      0.20      0.07       250



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# KNN
from sklearn.neighbors import KNeighborsClassifier
knn_classifier = KNeighborsClassifier(n_neighbors=3)
knn_classifier.fit(train_features_flat, train_labels)
knn_predictions = knn_classifier.predict(test_features_flat)

print('KNN Classification Report:')
print(classification_report(test_labels, knn_predictions))

KNN Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        50
           1       0.00      0.00      0.00        50
           2       0.00      0.00      0.00        50
           3       0.00      0.00      0.00        50
           4       0.20      1.00      0.33        50

    accuracy                           0.20       250
   macro avg       0.04      0.20      0.07       250
weighted avg       0.04      0.20      0.07       250



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
